In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5

import numpy as np
import matplotlib.pyplot as plt

from riptide import TimeSeries, ffa_search, find_peaks, SubIntegrations
from riptide.pipelines.pipeline import time_series_iterator
from riptide.clustering import cluster_1d

from riptide.pipelines.pipeline import Pipeline, DetectionCluster
from riptide.pipelines.candidate import Candidate


In [5]:
config_fname = '/home/vince/repositories/riptide/pipelines/example_config.yaml' 
input_dir = '/home/vince/work/pulsars/presto_time_series/J1855+0307/'
output_dir = '/home/vince/work/pulsars/presto_time_series/J1855+0307/'
pl = Pipeline(config_fname, input_dir, output_dir, pattern='*DM40*.inf')

In [6]:
pl.run()

2017-10-11,08:55:46.926: Computing Periodogram @ DM = 400.000
2017-10-11,08:55:51.984: Finding Peaks @ DM = 400.000
2017-10-11,08:55:52.672: New Detections: 21
2017-10-11,08:55:52.673: Total Detections: 21
2017-10-11,08:55:52.673: Clustering Detections ...
2017-10-11,08:55:52.674: Total Clusters: 21
2017-10-11,08:55:52.675: TimeSeries in RAM: 1
2017-10-11,08:55:52.691: Computing Periodogram @ DM = 404.000
2017-10-11,08:55:57.740: Finding Peaks @ DM = 404.000
2017-10-11,08:55:58.426: New Detections: 20
2017-10-11,08:55:58.427: Total Detections: 41
2017-10-11,08:55:58.427: Clustering Detections ...
2017-10-11,08:55:58.428: Total Clusters: 21
2017-10-11,08:55:58.429: TimeSeries in RAM: 2
2017-10-11,08:55:58.445: Computing Periodogram @ DM = 408.000
2017-10-11,08:56:03.493: Finding Peaks @ DM = 408.000
2017-10-11,08:56:04.176: New Detections: 19
2017-10-11,08:56:04.177: Total Detections: 60
2017-10-11,08:56:04.177: Clustering Detections ...
2017-10-11,08:56:04.178: Total Clusters: 23
2017-

In [7]:
for cl in pl.clusters:
    print(cl)
    print(cl.top_detection)

for ii, cand in enumerate(pl.candidates):
    print(ii, cand.metadata['best_period'])

DetectionCluster [size =    3, P0 = 1.275550079e+01, DM =  400.000, S/N =   7.90]
Detection [P0 = 1.275550079e+01, W =   9, DM =  400.000, S/N =   7.90]
DetectionCluster [size =    1, P0 = 8.453228951e+00, DM =  408.000, S/N =   6.55]
Detection [P0 = 8.453228951e+00, W =   3, DM =  408.000, S/N =   6.55]
DetectionCluster [size =    3, P0 = 7.608142853e+00, DM =  404.000, S/N =   7.03]
Detection [P0 = 7.608142853e+00, W =   1, DM =  404.000, S/N =   7.03]
DetectionCluster [size =    3, P0 = 7.478675365e+00, DM =  404.000, S/N =  10.16]
Detection [P0 = 7.478675365e+00, W =  42, DM =  404.000, S/N =  10.16]
DetectionCluster [size =    1, P0 = 6.763092041e+00, DM =  400.000, S/N =   6.55]
Detection [P0 = 6.763092041e+00, W =   2, DM =  400.000, S/N =   6.55]
DetectionCluster [size =    3, P0 = 5.917754173e+00, DM =  408.000, S/N =   7.53]
Detection [P0 = 5.917754173e+00, W =   3, DM =  408.000, S/N =   7.53]
DetectionCluster [size =    3, P0 = 5.072118282e+00, DM =  400.000, S/N =   7.60]


In [46]:
#cand = pl.candidates[17]
cube = cand.dpw_cube
dm_trials = np.sort(cand.dm_trials)

plt.figure(figsize=(14, 4))
plt.imshow(
    cube[:, :, 6],
    )
plt.colorbar(orientation='horizontal')
plt.tight_layout()
plt.show()


cand.subints.display(dpi=80)

In [14]:
class Const(object):
    kdm = 4.148808e3

def dispersion_delay(dm, fmin, fmax):
    """ Dispersion delay between two given frequencies """
    return Const.kdm * dm * abs(fmin**-2 - fmax**-2)

def dispersion_smearing_time(dm, freq, bw):
    return 2 * Const.kdm * dm * freq**-3 * bw

def dispersion_measure(delay, fmin, fmax):
    """ Dispersion measure as function of dispersion delay between two given frequencies """
    return delay / (Const.kdm * abs(fmin**-2 - fmax**-2))

def diagonal_dm(tsamp, freq, bw):
    """ DM at which the dispersion smearing time is equal to the sampling time """
    return tsamp / (2 * Const.kdm * freq**-3 * bw)

def select_dm_trials(trials, freq, bw, max_smear):
    """ Given a list of DM trials, select only those required to keep the total s"""

In [20]:
dispersion_smearing_time(100.0, 143.0, 48.0/3904)

0.0034888046178830367

In [19]:
dispersion_smearing_time(400.0, 1382.0, 400.0/1024)

0.0004911896752691573

In [26]:
det = pl.clusters[0][1]

In [28]:
det.snr_trials.shape

(250, 10)

In [25]:
pl.candidates[0].metadata

Metadata
{'analyst': 'Scott Ransom',
 'bandwidth': 400.0,
 'barycentered': True,
 'basename': 'J1855+0307_DM404.00',
 'bdiam': 981.0,
 'best_dm': 404.0,
 'best_period': 0.84535116,
 'best_snr': 18.1306,
 'best_width': 4,
 'breaks': False,
 'cbw': 0.390625,
 'decj': '03:02:38.8000',
 'dm': 404.0,
 'fbot': 1182.1953125,
 'instrument': '????',
 'mjd': 56937.40513565126,
 'nchan': 1024,
 'nsamp': 8754411,
 'observer': 'Unknown',
 'obstype': 'Radio',
 'raj': '18:55:04.4400',
 'skycoord': <SkyCoord (ICRS): (ra, dec) in deg
    ( 283.7685,  3.04411111)>,
 'source': 'G036.5+00.6_s',
 'source_name': 'G036.5+00.6_s',
 'telescope': 'Parkes',
 'tobs': 560.282304,
 'tsamp': 6.4e-05}

In [21]:
np.median([0,1,2,5,9,11,33221])

5.0

In [2]:
cand = Candidate.load_hdf5('/home/vince/pulsar.h5')

In [45]:
from matplotlib.gridspec import GridSpec

st = cand.dpw_cube
dt = np.sort(cand.dm_trials)
wt = cand.width_trials
pt = cand.period_trials

# Delta period in microseconds
dp = 1.0e6 * (pt - cand.metadata['best_period'])

# Delta period step
dp_step = np.diff(dp).mean()

# X-axis limits
xmin = dp[0]  - 0.5*dp_step
xmax = dp[-1] + 0.5*dp_step

gs = GridSpec(2, 1, height_ratios=(10, 1))

plt.figure(figsize=(10,5))
plt.subplot(gs[0, 0])
image = plt.imshow(
    st[:,:,6],
    extent=[dp[0]-0.5*dp_step, dp[-1]+0.5*dp_step, dt[0], dt[-1]],
    aspect='auto',
    origin='lower'
    )
plt.ylabel('DM', fontsize=12)
plt.xlabel('Trial Period - P0 (us)', fontsize=12)
plt.title(
    "P0 = {period:.6f} ms, DM = {dm:.3f}".format(period=1e3*cand.metadata['best_period'], dm=cand.metadata['best_dm'])
    )

plt.subplot(gs[1, 0])
cb = plt.colorbar(image, cax=plt.gca(), orientation='horizontal')
cb.set_label(label='S/N', size=12)

# plt.subplot(gs[1, 0])
# plt.plot(dp, st[:, self.iw], label='S/N at W = {0:d}'.format(self.width))
# plt.xlabel('Delta Period (us)', fontsize=12)
# plt.xlim(xmin, xmax)
# plt.grid(linestyle=':')
# plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

In [28]:
dt

array([ 26.54999924,  26.39999962,  26.35000038,  26.04999924,
        26.10000038,  26.14999962,  26.25      ,  26.20000076,
        26.29999924,  26.5       ,  24.79999924,  24.85000038,
        26.45000076,  25.25      ,  25.45000076,  25.        ,
        24.64999962,  24.70000076,  24.75      ,  25.14999962,
        25.89999962,  25.79999924,  25.95000076,  24.60000038,
        25.54999924,  25.85000038,  24.89999962,  24.35000038,
        24.95000076,  24.39999962,  25.20000076,  24.29999924,
        25.29999924,  25.39999962,  25.5       ,  24.20000076,
        25.04999924,  25.35000038,  25.60000038,  25.64999962,
        26.60000038,  25.10000038,  25.75      ,  25.70000076,
        24.54999924,  24.5       ,  24.45000076,  24.25      ,
        24.14999962,  24.10000038,  26.        ], dtype=float32)

In [53]:
cand.metadata

Metadata
{'analyst': 'sanidas',
 'bandwidth': 31.640621759999998,
 'barycentered': True,
 'basename': 'L611408_SAP0_BEAM20_DM25.35',
 'bdiam': 3600.0,
 'best_dm': 25.350000000000001,
 'best_period': 0.99586117,
 'best_snr': 13.335054,
 'best_width': 13,
 'breaks': True,
 'cbw': 0.012207030000000001,
 'decj': '52:04:43.0000',
 'dm': 25.350000000000001,
 'fbot': 119.433595369,
 'instrument': 'Unknown',
 'mjd': 58027.531233000489,
 'nchan': 2592,
 'nsamp': 7392000,
 'observer': 'unset',
 'obstype': 'Radio',
 'onoff': array([[7391999, 7391999],
       [      0, 7308040]]),
 'raj': '14:26:36.0000',
 'skycoord': <SkyCoord (ICRS): (ra, dec) in deg
    ( 216.65,  52.07861111)>,
 'source': 'LOTAAS-P1818C-SAP0',
 'source_name': 'LOTAAS-P1818C-SAP0',
 'telescope': 'LOFAR',
 'tobs': 3633.3157103508693,
 'tsamp': 0.00049151998246088602}